#  Phase 2 : Nettoyage, Normalisation et Préparation des Données (Data Engineering)

Après l'Analyse Exploratoire (EDA), cette étape vise à transformer les données brutes en un **Dataset Master** exploitable.

### Objectifs du pipeline :
1.  **Harmonisation Géographique** : Résoudre les problèmes d'accents et de casse pour garantir 100% de succès lors des jointures.
2.  **Typage des données** : Conversion des dates et des formats numériques.
3.  **Traitement des Outliers** : Nettoyage des délais de traitement aberrants.
4.  **Enrichissement (Feature Engineering)** : Création de variables temporelles pour l'analyse de la saisonnalité fine.

In [31]:
import pandas as pd
import numpy as np
import os
import unicodedata

In [32]:
def normalize_string(s):
    """
    Normalise les chaînes de caractères pour les jointures géographiques :
    - Supprime les accents (Unicode NFD)
    - Passage en majuscules
    - Suppression des espaces inutiles
    """
    if pd.isna(s) or not isinstance(s, str):
        return s
    # Normalisation Unicode pour séparer les caractères des accents, puis encodage ASCII pour les ignorer
    s = unicodedata.normalize('NFD', s).encode('ascii', 'ignore').decode("utf-8")
    return s.strip().upper()

In [33]:
# Chemin de base des fichiers
BASE_PATH = '/content/drive/MyDrive/projet datalab/data/test_analyste'

## 1. Chargement et Normalisation de surface
Nous appliquons la fonction `normalize_string` dès le chargement sur les colonnes 'region', 'prefecture' et 'commune' de TOUS les fichiers. C'est la clé pour éviter la perte de données lors des fusions.

In [34]:
files = {
    'demandes': 'demandes_service_public.csv',
    'centres': 'centres_service.csv',
    'logs': 'logs_activite.csv',
    'socio': 'donnees_socioeconomiques.csv',
    'communes': 'details_communes.csv',
    'routes': 'reseau_routier_togo_ext.csv',
    'dev': 'developpement.csv',
    'docs_hist': 'documents_administratifs_ext.csv'
}

In [35]:
dfs = {}
print("--- Chargement des fichiers ---")
for key, filename in files.items():
    path = os.path.join(BASE_PATH, filename)
    if os.path.exists(path):
        df = pd.read_csv(path)
        # Normalisation immédiate des colonnes clés
        for col in ['region', 'prefecture', 'commune']:
            if col in df.columns:
                df[col] = df[col].apply(normalize_string)
        dfs[key] = df
        print(f"{filename} chargé et normalisé.")

--- Chargement des fichiers ---
demandes_service_public.csv chargé et normalisé.
centres_service.csv chargé et normalisé.
logs_activite.csv chargé et normalisé.
donnees_socioeconomiques.csv chargé et normalisé.
details_communes.csv chargé et normalisé.
reseau_routier_togo_ext.csv chargé et normalisé.
developpement.csv chargé et normalisé.
documents_administratifs_ext.csv chargé et normalisé.


## 2. Typage et Imputation
Nous convertissons les dates au format standard Python et gérons les valeurs manquantes spécifiques aux logs opérationnels pour ne pas fausser les statistiques de rejet.

In [36]:
# 1. Traitement des Dates
for key in ['demandes', 'logs', 'centres']:
    if key in dfs:
        date_cols = [c for c in dfs[key].columns if 'date' in c]
        for col in date_cols:
            dfs[key][col] = pd.to_datetime(dfs[key][col], errors='coerce')

In [37]:
# 2. Imputation dans les Logs
if 'logs' in dfs:
    dfs['logs']['type_document'] = dfs['logs']['type_document'].fillna('NON SPECIFIE')
    dfs['logs']['raison_rejet'] = dfs['logs']['raison_rejet'].fillna('AUCUN')


## 3. Consolidation du Master Dataset
Nous fusionnons les demandes avec les indicateurs socio-économiques et de développement. Ce dataset "plat" permettra des analyses de corrélation (ex: Revenu vs Délai).

In [38]:
# Fusion des référentiels géographiques (Communes + Socio + Dev)
geo_ref = pd.merge(dfs['communes'], dfs['socio'], on=['commune', 'prefecture', 'region'], how='outer')
geo_ref = pd.merge(geo_ref, dfs['dev'], on=['commune', 'prefecture', 'region'], how='outer')

# Création du Master Dataset à partir des demandes
master_df = pd.merge(dfs['demandes'], geo_ref, on=['commune', 'prefecture', 'region'], how='left')

# Vérification du succès de la jointure
loss = master_df['population'].isna().sum()
print(f" Lignes sans correspondance socio-éco : {loss} sur {len(master_df)}")

 Lignes sans correspondance socio-éco : 24 sur 600


In [39]:
master_df.head()

,demande_id,region,prefecture,commune,quartier,type_document,categorie_document,nombre_demandes,delai_traitement_jours,taux_rejet,...,taux_pauvrete,acces_electricite,acces_eau_potable,acces_internet,indice_developpement,score_education,score_sante,nombre_ecoles,nombre_hopitaux,nombre_banques
0,D001,CENTRALE,SOTOUBOUA,SOTOUBOUA,Nyékonakpoé,Carte d'identité,Identité,168,1,0.00,...,0.43,0.79,0.76,0.32,0.55,0.61,0.59,74.0,4.0,11.0
1,D002,KARA,BINAH,PAGOUDA,Hôpital,Carte d'identité,Identité,17,22,0.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,D003,PLATEAUX,OGOU,ATAKPAME,Bè,Certificat de nationalité,Identité,104,4,0.08,...,0.34,0.89,0.86,0.44,0.65,0.71,0.66,125.0,9.0,23.0
3,D004,SAVANES,OTI,MANGO,Amoutivé,Carte d'identité,Identité,92,39,0.02,...,0.51,0.73,0.71,0.29,0.51,0.56,0.54,72.0,4.0,12.0
4,D005,MARITIME,ZIO,TSEVIE,Adidogomé,Livre de famille,Civil,138,7,0.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 4. Raffinement et Feature Engineering
Pour assurer la fiabilité des statistiques, nous traitons les valeurs aberrantes des délais et extrayons des informations temporelles (jours ouvrés vs week-end).

In [40]:
#Gestion des Outliers
if 'delai_traitement_jours' in master_df.columns:
    # On élimine les erreurs (délais négatifs)
    master_df = master_df[master_df['delai_traitement_jours'] >= 0]
    # Plafonnement au 99ème percentile pour neutraliser les erreurs de saisie extrêmes
    upper_limit = master_df['delai_traitement_jours'].quantile(0.99)
    master_df['delai_traitement_jours'] = master_df['delai_traitement_jours'].clip(upper=upper_limit)
    print(f" Délais de traitement plafonnés à {upper_limit} jours (99e percentile).")



 Délais de traitement plafonnés à 45.0 jours (99e percentile).


In [41]:
master_df.head()

,demande_id,region,prefecture,commune,quartier,type_document,categorie_document,nombre_demandes,delai_traitement_jours,taux_rejet,...,taux_pauvrete,acces_electricite,acces_eau_potable,acces_internet,indice_developpement,score_education,score_sante,nombre_ecoles,nombre_hopitaux,nombre_banques
0,D001,CENTRALE,SOTOUBOUA,SOTOUBOUA,Nyékonakpoé,Carte d'identité,Identité,168,1,0.00,...,0.43,0.79,0.76,0.32,0.55,0.61,0.59,74.0,4.0,11.0
1,D002,KARA,BINAH,PAGOUDA,Hôpital,Carte d'identité,Identité,17,22,0.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,D003,PLATEAUX,OGOU,ATAKPAME,Bè,Certificat de nationalité,Identité,104,4,0.08,...,0.34,0.89,0.86,0.44,0.65,0.71,0.66,125.0,9.0,23.0
3,D004,SAVANES,OTI,MANGO,Amoutivé,Carte d'identité,Identité,92,39,0.02,...,0.51,0.73,0.71,0.29,0.51,0.56,0.54,72.0,4.0,12.0
4,D005,MARITIME,ZIO,TSEVIE,Adidogomé,Livre de famille,Civil,138,7,0.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
#  Variables Temporelles
if 'date_demande' in master_df.columns:
    master_df['mois'] = master_df['date_demande'].dt.month
    master_df['nom_mois'] = master_df['date_demande'].dt.month_name()
    master_df['jour_semaine'] = master_df['date_demande'].dt.day_name()
    master_df['est_weekend'] = master_df['date_demande'].dt.dayofweek >= 5

In [43]:
master_df.head()

,demande_id,region,prefecture,commune,quartier,type_document,categorie_document,nombre_demandes,delai_traitement_jours,taux_rejet,...,indice_developpement,score_education,score_sante,nombre_ecoles,nombre_hopitaux,nombre_banques,mois,nom_mois,jour_semaine,est_weekend
0,D001,CENTRALE,SOTOUBOUA,SOTOUBOUA,Nyékonakpoé,Carte d'identité,Identité,168,1,0.00,...,0.55,0.61,0.59,74.0,4.0,11.0,9,September,Saturday,True
1,D002,KARA,BINAH,PAGOUDA,Hôpital,Carte d'identité,Identité,17,22,0.09,...,NaN,NaN,NaN,NaN,NaN,NaN,12,December,Friday,False
2,D003,PLATEAUX,OGOU,ATAKPAME,Bè,Certificat de nationalité,Identité,104,4,0.08,...,0.65,0.71,0.66,125.0,9.0,23.0,8,August,Sunday,True
3,D004,SAVANES,OTI,MANGO,Amoutivé,Carte d'identité,Identité,92,39,0.02,...,0.51,0.56,0.54,72.0,4.0,12.0,6,June,Thursday,False
4,D005,MARITIME,ZIO,TSEVIE,Adidogomé,Livre de famille,Civil,138,7,0.12,...,NaN,NaN,NaN,NaN,NaN,NaN,2,February,Thursday,False


## 5. Exportation des Datasets Finaux
Nous générons deux fichiers :
1. `master_data_cleaned.csv` : Pour l'analyse globale et territoriale.
2. `performance_centres_cleaned.csv` : Pour le monitoring spécifique des centres de service.

In [44]:
# Dataset de performance spécifique
perf_df = pd.merge(dfs['logs'], dfs['centres'], on='centre_id', how='inner')

# Sauvegarde locale
master_df.to_csv('master_data_cleaned.csv', index=False)
perf_df.to_csv('performance_centres_cleaned.csv', index=False)

print("\n" + "="*50)
print("NETTOYAGE TERMINÉ ET FICHIERS SAUVEGARDÉS")
print(f"Dimensions Master : {master_df.shape}")
print(f"Dimensions Performance : {perf_df.shape}")
print("="*50)


NETTOYAGE TERMINÉ ET FICHIERS SAUVEGARDÉS
Dimensions Master : (600, 50)
Dimensions Performance : (450, 29)


In [45]:
perf_df.head()

,log_id,centre_id,date_operation,type_operation,type_document,nombre_traite,delai_effectif,nombre_rejete,raison_rejet,personnel_present,...,quartier,latitude,longitude,personnel_capacite_jour,nombre_guichets,heures_ouverture,horaire_nuit,equipement_numerique,date_ouverture,statut_centre
0,LOG001,CT015,2023-10-30,Traitement,Carte d'identité,241,41,31,Non éligible,15,...,Carrefour,7.5719,0.8589,74,2,08:00-16:00,Non,Partiel,2017-06-14,Actif
1,LOG002,CT026,2023-04-21,Maintenance,NON SPECIFIE,0,0,0,AUCUN,12,...,Résidentiel,8.9819,0.8310,345,11,07:30-17:30,Non,Complet,2020-10-16,Actif
2,LOG003,CT049,2023-08-22,Traitement,Livre de famille,62,19,0,AUCUN,12,...,Carrefour,9.9226,1.1138,77,5,08:00-16:00,Non,Partiel,2018-09-28,Actif
3,LOG004,CT047,2023-07-13,Traitement,Acte de naissance,139,22,12,Double demande,7,...,Carrefour,9.9934,0.8713,74,4,08:00-16:00,Non,Partiel,2023-04-10,Actif
4,LOG005,CT008,2023-04-08,Traitement,Passeport,71,35,6,Double demande,13,...,Préfecture,7.6305,0.9746,355,11,07:30-17:30,Non,Complet,2017-07-12,Actif


### Analyse des résultats du traitement
Le pipeline de nettoyage et de fusion a produit des résultats hautement satisfaisants :

1. **Efficacité du Matching** : Le taux de correspondance géographique est de **96%**. Seules 24 lignes (4%) n'ont pas trouvé de correspondance socio-économique, ce qui est marginal et n'altère pas la représentativité statistique globale.
2. **Enrichissement de données** : Le passage à **50 variables** dans le fichier Master permet désormais des corrélations multidimensionnelles (ex: impact de l'accès à l'électricité sur le délai de traitement).
3. **Consolidation opérationnelle** : Le dataset de performance (450 entrées) est prêt pour un monitoring précis de la charge de travail par centre.

**Conclusion :** Les données sont désormais "Analytics-Ready". Nous pouvons passer à la formulation des recommandations stratégiques.